In [17]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from utilities import *


## Loading The data

In [18]:
#Load signal 
data = pd.read_csv('chbmit_preprocessed_data.csv')
data.columns
grouped_data = data.groupby('Outcome')
preictal_data = grouped_data.get_group(0)
ictal_data = grouped_data.get_group(1)
#  data exploration 
print(preictal_data.columns)
print(preictal_data.shape)
print(ictal_data.shape)
print(preictal_data.shape[0]/(256*60)) # print length of recording in minutes, 256 is sampling frequency

Index(['# FP1-F7', 'C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7',
       'F8-T8', 'FP1-F3', 'FP2-F4', 'FP2-F8', 'FT10-T8', 'FT9-FT10', 'FZ-CZ',
       'P3-O1', 'P4-O2', 'P7-O1', 'P7-T7', 'P8-O2', 'T7-FT9', 'T7-P7',
       'T8-P8-0', 'T8-P8-1', 'Outcome'],
      dtype='object')
(1048575, 24)
(1048575, 24)
68.2666015625


## Preprocessing The data

In [19]:
# create trials split data to train and test
subsetted_preictal, labels_preictal = subset_trials(preictal_data,1)
subsetted_ictal, labels_ictal = subset_trials(ictal_data,1)
all_trials = np.concatenate((subsetted_preictal,subsetted_ictal), axis=0) # 136 * 15360 * 23
all_labels = np.concatenate((labels_preictal,labels_ictal), axis=0) #Zero is preictal, 1 is ictal

## Classifier Performance on CV data with no selection, and with different methods of selection/reduction 

In [20]:
# features = np.concatenate((freq_features(all_trials),stat_features(all_trials)), axis=1)
X_train, X_test, Y_train, Y_test = split_data(extract_features(all_trials),all_labels, 0.4)
trainer = SVC(kernel='sigmoid')
trainer.fit(X_train,Y_train)
print(accuracy_score(Y_test,trainer.predict(X_test)))
# -------------------------
X_test, X_cv, y_test, y_cv = split_data(X_test, Y_test, 0.5, False)

0.9272727272727272


In [21]:
X_train_reduced, X_test_reduced, X_cv_reduced = mutual_info(X_train, X_test, X_cv, Y_train,10)
trainer = SVC(kernel='sigmoid')
trainer.fit(X_train_reduced,Y_train)
print(accuracy_score(y_cv,trainer.predict(X_cv_reduced)), end=' ')

0.9642857142857143 

In [22]:
X_train_reduced, X_test_reduced, X_cv_reduced = pca(X_train, X_test, X_cv, 0.85)

trainer = SVC(kernel='sigmoid')
trainer.fit(X_train_reduced,Y_train)
print(accuracy_score(y_cv,trainer.predict(X_cv_reduced)), end=' ')

0.9285714285714286 

In [23]:
X_train_reduced, X_test_reduced, X_cv_reduced = wilcoxon_test(X_train, X_test, X_cv, Y_train,0.005)

trainer = SVC(kernel='sigmoid')
trainer.fit(X_train_reduced,Y_train)
print(accuracy_score(y_cv,trainer.predict(X_cv_reduced)), end=' ')

0.9642857142857143 

### We can notice that selection and reduction methods reduced total time of learning and testing by a factor of 10, while maintaining the accuracy or increasing it.